In [2]:
from os import environ
from pathlib import Path
from json import loads, dumps
environ["OPENAI_API_KEY"] = Path("~/.openaiapikey").expanduser().read_text().strip()
from openai import OpenAI
client = OpenAI()

In [16]:
testFunction = { # parameters json schema definition : https://json-schema.org/overview/what-is-jsonschema
    "name" : "function",
    "description" : "Give me a summary of the following text",
    "parameters" : { 
        "type" : "object",
        "properties" : {
            "topic" : {
                "type" : "string",
                "description" : "The topic of the text"
            },
            "protagonist" : {
                "type" : "string",
                "description" : "The protagonist of the text"
            },
            "context" : {
                "type" : "string",
                "description" : "The general context of the text"
            },
            "duration":
            {
                "type" : "number",
                "description" : "The duration of the action in hours"
            },
            "involved items":
            {
                "type" : "array",
                "items" : {
                    "type" : "object",
                    "properties" : {
                        "name" : {
                            "type" : "string",
                            "description" : "The name of the item"
                        },
                        "quantity" : {
                            "type" : "number",
                            "description" : "The quantity of the item"
                        }
                    }
                },
                "description" : "The items involved in the action"
            },
        },
    }
}
testText = "Yesterday I went to the store and bought some apples. I also bought some oranges. I then went home and made a fruit salad. It was delicious."

In [31]:
openaiClient = OpenAI()
def completion(query, format):
    answer = openaiClient.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
                "role": "user",
                "content": query
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "response",
                "strict": True,
                "schema" : format
            }
        }
    )
    if answer.choices[0].message.refusal != None:
        raise Exception(answer.choices[0].message.refusal)
    return answer.choices[0].message.content


In [33]:
testFunction2 = {
        "type" : "object",
        "properties" : {
            "list" : {
                "type" : "array",
                "items" : {
                    "type" : "string"
                }
            }
        },
        "required" : ["list"],
        "additionalProperties" : False
    }
prompt2 = "Describe the purpose of a university"

In [34]:
completion(prompt2, testFunction2)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid schema for response_format 'response': In context=('properties', 'list'), 'minItems' is not permitted", 'type': 'invalid_request_error', 'param': 'response_format', 'code': None}}

In [6]:
def gpt3_5_FunctionCall(prompt, format):
    restult = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {"role":"user", "content":prompt}
        ],
        functions=[
            {"name":"function", 
             "description":prompt, 
             "parameters":{
                    "type":"object",
                    "properties":{
                        "result":format
                    }
                }
            }
        ],
        function_call="auto",
    ).choices[0].message.function_call.arguments
    return loads(restult)["result"]

In [9]:
gpt3_5_FunctionCall(prompt2, testFunction2)

[['Higher Education',
  'Universities provide advanced education and training in various fields, preparing students for specific careers and fostering intellectual growth.'],
 ['Research and Development',
  'Universities serve as centers for research and innovation, contributing to knowledge creation and addressing societal challenges.'],
 ['Cultural Exchange',
  'Universities promote cultural diversity by bringing together students from various backgrounds and facilitating exchange of ideas and perspectives.'],
 ['Community Engagement',
  'Universities often engage with local communities through outreach programs, enhancing social responsibility and providing services.'],
 ['Personal Development',
  'Universities help students develop critical thinking, problem-solving skills, and personal integrity, shaping well-rounded individuals.']]